# Scikit Learn - Extra Trees Classifier

First, import numpy, cifar10 from `keras.datasets`, and the `ExtraTreesClassifier` from sklearn.

In [1]:
import numpy as np
from keras.datasets import cifar10
from sklearn.ensemble import ExtraTreesClassifier

Using TensorFlow backend.


Next, we import and format our data.

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255
x_test = x_test / 255

m_train, m_test = x_train.shape[0], x_test.shape[0]

x_train, x_test = x_train.reshape(m_train, -1), x_test.reshape(m_test, -1)

x_train = x_train / np.linalg.norm(x_train, ord=2, axis=1, keepdims=True)
x_test = x_test / np.linalg.norm(x_test, ord=2, axis=1, keepdims=True)

170500096/170498071 [==============================] - 19s 0us/step


Then. we initialize and train our classifier.

In [3]:
clf = ExtraTreesClassifier()

clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

And finally, test it out!

In [4]:
count = 0
predictions = clf.predict(x_test)

for i in range(len(y_test)):
  if(y_test[i]==predictions[i]):
    count+=1
    

print((count/len(y_test))*100)

35.19


You can see that this is a disappointingly low score. In the next section we will take a different approach to increase this.

# Keras and Tensorflow

In order to make a simple model with Keras and Tensorflow, we will import tensorflow and some things from keras as well.

In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.constraints import maxnorm
from PIL import Image
from keras.utils import np_utils
from keras.datasets import cifar10
import numpy as np

Now, load in the data.


In [0]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

new_X_train = x_train.astype('float32')
new_X_test = x_test.astype('float32')
new_X_train /= 255 
new_X_test /= 255
new_Y_train = np_utils.to_categorical(y_train)
new_Y_test = np_utils.to_categorical(y_test)

Next, we create our model, compile it, and fit it to the data.

In [11]:
model = Sequential([
    Conv2D(32,(3,3),input_shape=(32,32,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),input_shape=(32,32,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(10,activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(new_X_train,new_Y_train,epochs=3,batch_size=32)

Epoch 1/3
50000/50000 [==============================] - 14s 279us/step - loss: 1.4529 - acc: 0.4839
Epoch 2/3
50000/50000 [==============================] - 19s 373us/step - loss: 1.1091 - acc: 0.6159
Epoch 3/3
50000/50000 [==============================] - 13s 261us/step - loss: 0.9878 - acc: 0.6576


Now, we will store some data that allows us to use the model without needing to train it again.

In [0]:
json_string = model.to_json()
model.save_weights('model_weights.h5')

Now, we load in our model using the data we stored.

In [0]:
from keras.models import model_from_json

model = model_from_json(json_string)
model.load_weights('model_weights.h5')
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Finally, we test out this model!

In [14]:
count = 0
predictions = model.predict(x_test)

for i in range(len(y_test)):
  if(y_test[i]==np.argmax(predictions[i])):
    count+=1
    

print((count/len(y_test))*100)

49.5


This is a significant increase in accuracy. If you want an even higher accuracy, train the model for more epochs, play around with hyperparameters, and add some layers!